In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import os
import time
import random

# Load the Excel sheet with ticker and date information
excel_file = 'C:\\Users\\LENOVO\\OneDrive\\Desktop\\Book2.xlsx'
df_tickers = pd.read_excel(excel_file, sheet_name='Sheet1')  # Adjust sheet name if needed

# Function to fetch OHLC data for a given ticker and date
def fetch_ohlc_data(ticker, start_date, end_date):
    try:
        # Fetch data including Saturdays and Sundays
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        # Filter out Saturdays (5) and Sundays (6)
        stock_data = stock_data[stock_data.index.dayofweek < 5]
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker} on {start_date}: {e}")
        return None

# Function to plot and save OHLC chart with volume bars
def plot_save_ohlc_chart(ticker, date, ohlc_data, output_folder):
    if ohlc_data is None:
        return  # Skip plotting if there was an error fetching data
    
    # Convert date to matplotlib date format
    ohlc_data['Date'] = mdates.date2num(ohlc_data.index)

    # Increase the size of the chart
    fig = plt.figure(figsize=(16, 10))
    fig.patch.set_facecolor('#121212')  # Background color

    # Create a 2x1 grid
    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])

    # Plot the candlestick chart in the upper subplot
    ax1 = plt.subplot(gs[0])
    candlestick_ohlc(ax1, ohlc_data[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.7, colorup='#089981', colordown='#f23645')
    
    # Plot 21-day EMA
    ema_21 = ohlc_data['Close'].ewm(span=21, adjust=False).mean()
    ax1.plot(ohlc_data['Date'], ema_21, label='21 EMA', color='orange', linewidth=1)
    
    # Plot 50-day EMA
    ema_50 = ohlc_data['Close'].ewm(span=50, adjust=False).mean()
    ax1.plot(ohlc_data['Date'], ema_50, label='50 EMA', color='teal', linewidth=1)

    # Format the x-axis to show dates nicely
    ax1.xaxis_date()
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    # Rotate x-axis labels for better readability
    plt.setp(ax1.get_xticklabels(), rotation=0)

    # Set labels and title for the upper subplot
    ax1.set_xlabel('Date', color='white')  # White color for X-axis label
    ax1.set_ylabel('Price', color='white')  # White color for Y-axis label
    ax1.set_title(f'OHLC Chart for {ticker} - {date}', color='white')  # White color for title
    ax1.legend()

    # Set grid, dark mode colors, and reduce opacity of gridlines
    ax1.grid(True, linestyle='--', alpha=0.3, color='#555555')  # Reduced opacity for grid lines
    ax1.spines['bottom'].set_color('white')  # White color for X-axis
    ax1.spines['top'].set_color('white')  # White color for X-axis
    ax1.spines['right'].set_color('white')  # White color for Y-axis
    ax1.spines['left'].set_color('white')  # White color for Y-axis
    ax1.tick_params(axis='both', colors='white')  # White color for ticks

    # Create the lower subplot for volume bars
    ax2 = plt.subplot(gs[1], sharex=ax1)
    
    # Plot volume bars as columns
    ax2.bar(ohlc_data['Date'], ohlc_data['Volume'], color='black', alpha=1, width=0.7)

    # Set labels and title for the lower subplot
    ax2.set_xlabel('Date', color='white')  # White color for X-axis label
    ax2.set_ylabel('Volume', color='white')  # White color for Y-axis label
    ax2.tick_params(axis='both', colors='white')  # White color for ticks
    
    # Highlight the candlestick of the initial input date
    initial_date_index = ohlc_data.index.get_loc(date)
    ax1.axvline(x=ohlc_data['Date'][initial_date_index], color='orange', linestyle='solid', linewidth=5, alpha=0.1, label='Initial Date')
    
    try:
        # Save the plot as a PNG file with formatted date information in the filename
        formatted_date = date.strftime('%Y-%m-%d_%H-%M-%S')
        output_file = f'{output_folder}/{ticker}_{formatted_date}_ohlc_chart.png'
        fig.savefig(output_file, bbox_inches='tight')
        plt.close()
    except OSError as e:
        print(f"Error saving file for {ticker} on {date}: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False
    
    return True

# Specify the output folder
output_folder = 'output_charts'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through each row in the DataFrame
for index, row in df_tickers.iterrows():
    ticker = row['Ticker']
    date = row['Date']

    # Fetch OHLC data for 60 days before and after the specified date
    start_date = pd.to_datetime(date) - pd.DateOffset(days=90)
    end_date = pd.to_datetime(date) + pd.DateOffset(days=370)
    
    print(f"Fetching data for {ticker} on {date}...")
    
    # Introduce a random delay between 1 and 4 seconds
    delay = random.uniform(1, 4)
    time.sleep(delay)
    
    # Plot and save OHLC chart with volume bars
    ohlc_data = fetch_ohlc_data(ticker, start_date, end_date)
    if ohlc_data is not None:
        print(f"Data fetched successfully for {ticker} on {date}.")
        try:
            # Try to locate the timestamp in the index
            initial_date_index = ohlc_data.index.get_loc(date)
        except KeyError:
            print(f"Timestamp {date} not found in DataFrame. Skipping...")
            continue  # Continue with the next iteration of the loop

        try:
            if not plot_save_ohlc_chart(ticker, date, ohlc_data, output_folder):
                print(f"Failed to save chart for {ticker} on {date}.")
        except Exception as plot_error:
            print(f"Error plotting and saving chart for {ticker} on {date}: {plot_error}")
            continue  # Continue with the next iteration of the loop
    else:
        print(f"Skipping {ticker} on {date} due to data fetch error.")

print('OHLC charts with volume bars have been generated and saved in the output folder.')


Fetching data for DIACABS.NS on 2016-03-30 00:00:00...
[*********************100%%**********************]  1 of 1 completed
Data fetched successfully for DIACABS.NS on 2016-03-30 00:00:00.
Fetching data for SEJALLTD.NS on 2016-04-13 00:00:00...
[*********************100%%**********************]  1 of 1 completed
Data fetched successfully for SEJALLTD.NS on 2016-04-13 00:00:00.
Fetching data for SEJALLTD.NS on 2016-05-10 00:00:00...
[*********************100%%**********************]  1 of 1 completed
Data fetched successfully for SEJALLTD.NS on 2016-05-10 00:00:00.
Fetching data for RADAAN.NS on 2016-05-12 00:00:00...
[*********************100%%**********************]  1 of 1 completed
Data fetched successfully for RADAAN.NS on 2016-05-12 00:00:00.
Fetching data for SEJALLTD.NS on 2016-05-12 00:00:00...
[*********************100%%**********************]  1 of 1 completed
Data fetched successfully for SEJALLTD.NS on 2016-05-12 00:00:00.
Fetching data for KAUSHALYA.NS on 2016-05-13 00:00:

KeyboardInterrupt: 